In [1]:
import os, re, sys
import nltk
from nltk import FreqDist
import pymongo
import collections
from com.ibm.watson import DESKTOP, DOCS_DIR
import com.ibm.watson.utils.pubmed_utils as pu

In [8]:
base_dir = os.path.join(DOCS_DIR, "Curation", "pubmed")
pmids = []
pmid_regex = re.compile("\d{5,}")
with open(os.path.join(base_dir, "temp.txt")) as f:
    for line in f:
        pmids.extend(pmid_regex.findall(line.strip()))
    
pmids = set(pmids)
print len(pmids)
with open(os.path.join(base_dir, "pmids.txt"), "wb") as f:
    [f.write(pmid.strip() + "\n") for pmid in pmids]

812


In [9]:
medline_articles = pu.get_medline(list(pmids))
    
output_f = os.path.join(base_dir, "pubmed_mesh_terms.tsv")
substance_regex = re.compile("(\()(.*)(\))")
with open(output_f, "wb") as f:
    for article in medline_articles:
        pmid = article.get("PMID")
        substances = [ match.group(2) for match in [substance_regex.search(substance) for substance in article.get("RN", [])] if match ]
        to_write = {"Mesh Term" : article.get("MH", []), "Publication Types" : article.get("PT", []), "Substances" : substances}
        for (key, values) in to_write.items():
            for value in values:
                #print "\t".join([ pmid, key, value, "\n" ])
                f.write("\t".join([ pmid, key, value, "\n" ]))

In [3]:
pmids = []
func_status = []
with open(os.path.join(DESKTOP, "functional_study", "geneFilters.tsv")) as f:
    for raw in f:
        raw = raw.strip().split("\t")
        pmid = raw[-3].strip()
        pmids.append(pmid)
        func_status.append(raw[-1].strip())
pmids.pop(0)
func_status.pop(0)
print FreqDist(func_status).most_common(2)

medline_articles = pu.get_medline(pmids)
mesh_terms_func = []
mesh_term_nonfunc = []
regex = re.compile("(I|We|we)\s+(\w+)")
regex1 = re.compile("In this \w+")
all_verbs = []
for i, article in enumerate(medline_articles):
    #print "\n" + article.get("PMID") + ": "
    mesh_terms = article.get("MH") if "MH" in article else []
    mesh_terms_func.extend(mesh_terms)
    status = func_status[i]
    if status == "Functional Study":
        mesh_terms_func.extend(mesh_terms)
    else:
        mesh_term_nonfunc.extend(mesh_terms)
        
    abstract = article.get("AB")
    sentences = nltk.sent_tokenize(abstract)
    last_five = sentences[-5:]
    sents_of_interest = [ sent for sent in last_five if ((True if regex.search(sent) else False) + (True if regex1.search(sent) else False) > 0) ]
    tags = [ nltk.pos_tag(words) for words in [ nltk.word_tokenize(sent) for sent in sents_of_interest ] ]
    #print tags
    #verbs = [ pair[1] for pair in sent_tags for sent_tags in tags ]
    for sent_tags in tags:
        verbs = [ word for (word, pos) in sent_tags if pos.startswith("VB") ]
        #print verbs
        all_verbs.extend(verbs)
    #print sents_of_interest
        
fdist1 = FreqDist(mesh_terms_func)
fdist2 = FreqDist(mesh_term_nonfunc)
fdist3 = FreqDist(all_verbs)
#print fdist1.most_common(25)
#print fdist2.most_common(25)
#print fdist3.most_common(25)

[('Functional Study', 93), ('Non-Functional Study', 60)]


In [24]:
regex = re.compile("(\w+)(\(\w+\))\s+(\w+)\s+(\w+)(\(\w+\))")
text = "expression(Subcellular_Function_Event)	Target	FAS(Protein_Specific_Entity)"
match = regex.search(text)
if match:
    print match.group(1)
    print match.group(3)
    print match.group(4)

expression
Target
FAS


In [26]:
all_relations = []
with open(os.path.join(DESKTOP, "functional_study", "genesWithTs7.tsv")) as f:
    for i, raw in enumerate(f):
        line = raw.strip().split("\t")
        #print line
        relations = line[10:]
        #for relation in relations:
        match = regex.search(raw.strip())
        if match:
            all_relations.append(match.group(2)[1:-1] + "->" + match.group(3) + "->" + match.group(5)[1:-1])
                    
fdist1 = FreqDist(all_relations)
for (rel, count) in fdist1.most_common(10):
    print "%s:  %d" % (rel, count)

Subcellular_Function_Event->Target->Protein_Specific_Entity:  87
HCLS_Activity_Event->Target->Protein_Specific_Entity:  14
Cause_Change_Event->Agent->Protein_Specific_Entity:  6
Evidence_Event->Target->Protein_Specific_Entity:  5
Subcellular_Function_Event->Agent->Protein_Specific_Entity:  5
Protein_Specific_Entity->Has_Attribute->Nucleic_Acid_Nucleoside_or_Nucleotide_Entity:  5
Protein_Specific_Entity->Has_Attribute->Protein_Component_Entity:  5
Cell_Entity->Has_Attribute->Protein_Specific_Entity:  3
Creation_Event->Target->Protein_Specific_Entity:  3
Subcellular_Function_Event->Target->Gene_or_Genome_Entity:  3
